In [1]:
import qrcode
import tkinter.messagebox
import requests
import json
import time
from PIL import Image,ImageTk
import tkinter as tk
import os
import re
from tkinter import ttk
from bs4 import BeautifulSoup
from tkinter import font

font=('song ti',12)

def code():
    global flag
    flag=True
    headers = {
        'User-Agent': 'apifox/1.0.0 (https://www.apifox.cn)',
        'Accept': '*/*',
        'Host': 'passport.bilibili.com',
        'Connection': 'keep-alive'
    }
    global qrimg,key,face
    url = "http://passport.bilibili.com/x/passport-login/web/qrcode/generate"
    response = requests.request("GET", url, headers=headers)
    res_json=json.loads(response.text)
    if res_json['code']==0:
        url=res_json['data']['url']
        key=res_json['data']['qrcode_key']
        qrimg=qrcode.make(url).resize((300,300))
        qrimg=ImageTk.PhotoImage(qrimg)
        face.configure(image=qrimg)
    
def login():
    headers = {
        'User-Agent': 'apifox/1.0.0 (https://www.apifox.cn)',
        'Accept': '*/*',
        'Host': 'passport.bilibili.com',
        'Connection': 'keep-alive'
    }
    global key,cookie
    url = "http://passport.bilibili.com/x/passport-login/web/qrcode/poll?qrcode_key="+key
    while True:
        session = requests.Session()
        response = session.get(url, headers=headers)
        res_json=json.loads(response.text)
        if res_json['data']['code']==0:
            tkinter.messagebox.showinfo(title='成功',message='登录成功')
            session.post(res_json['data']['url'], headers=headers)
            cookie = requests.utils.dict_from_cookiejar(session.cookies)
            with open('cookie.txt','w')as f:
                f.write(json.dumps(cookie))
            break
        time.sleep(5)

def search(key,one=False):
    headers = {
        'User-Agent': 'apifox/1.0.0 (https://www.apifox.cn)',
        'Accept': '*/*',
        'Host': 'api.bilibili.com',
        'Connection': 'keep-alive'
    }
    url = 'http://api.bilibili.com/x/web-interface/search/type?search_type=video&keyword='+key+'&duration=1&tids=3'
    response=requests.request("GET", url, headers=headers,cookies=cookie).text
    result=json.loads(response)
    search_results=[]
    if one:
        i=result['data']['result'][0]
        i['title']=re.sub(r'<.*?>', "", i['title'])
        search_results.append([str(i['id']),i['title'],i['pic']])
    else:
        for i in result['data']['result']:
            i['title']=re.sub(r'<.*?>', "", i['title'])
            search_results.append([str(i['id']),i['title'],i['pic']])
    return search_results

def download_img(url,file_name):
    if not os.path.exists('./img/'+file_name):
        headers = {
        'User-Agent': 'apifox/1.0.0 (https://www.apifox.cn)',
        'Accept': '*/*',
        'Host': 'i0.hdslb.com',
        'Connection': 'keep-alive'
        }
        response = requests.request("GET", 'https:'+url, headers=headers).content
        if not os.path.exists('./img'):
            os.mkdir('./img')
        with open('./img/'+file_name+'.jpg','wb')as f:
            f.write(response)

def user():
    global mid
    url = "http://api.bilibili.com/x/space/myinfo"
    headers = {
    'User-Agent': 'apifox/1.0.0 (https://www.apifox.cn)',
    'Accept': '*/*',
    'Host': 'api.bilibili.com',
    'Connection': 'keep-alive'
    }
    response = requests.request("GET", url, headers=headers,cookies=cookie)
    with open('face.jpg','wb')as f:
        f.write(requests.get(json.loads(response.text)['data']['face']).content)
    mid=str(json.loads(response.text)['data']['mid'])
    return json.loads(response.text)['data']['name']

def change(key,j):
    global select_photo,select_photos
    values=[]
    img_url=[]
    av=[]
    for all_result in search(key):
        av.append(all_result[0])
        values.append(all_result[1])
        img_url.append(all_result[2])
    cbox['value'] = values
    cbox.current(0)
    def func(event):
        global select_photo,select_photos
        i=values.index(cbox.get())
        download_img(img_url[i],av[i])
        select_photo = Image.open('./img/'+av[i]+'.jpg').resize((256,144))
        select_photo=ImageTk.PhotoImage(select_photo)
        select_photos.append(select_photo)
        button[j].configure(image=select_photos[-1])
        lable[j]['text']=values[i]
        avs[j]=av[i]
        
    cbox.bind("<<ComboboxSelected>>",func)

def favor(avs,fid,cookie):
    headers = {
        'User-Agent': 'apifox/1.0.0 (https://www.apifox.cn)',
        'Accept': '*/*',
        'Host': 'api.bilibili.com',
        'Connection': 'keep-alive'
    }
    for av in avs:
        requests.post('https://api.bilibili.com/x/v3/fav/resource/deal?rid='+str(av)+'&type=2&csrf='+cookie['bili_jct']+'&add_media_ids='+str(fid)+'&jsonp=jsonp&platform=web',headers=headers,cookies=cookie).text

def song_list():
    global keys
    url = expression.get()
    if not url:
        url="https://music.163.com/playlist?id=403365085"
    headers = {
    'User-Agent': 'apifox/1.0.0 (https://www.apifox.cn)',
    'Cookie': 'os=pc',
    'Accept': '*/*',
    'Host': 'music.163.com',
    'Connection': 'keep-alive'
    }

    response = requests.request("GET", url, headers=headers).text

    bs = BeautifulSoup(response)
    songs=[]
    for song in bs.find_all(href=re.compile('/song\?id=\d')):
        song_name=re.sub(r'\（.*?\）','',song.text)
        song_name=re.sub(r'\(.*?\)','',song_name)
        song_name=re.sub(r'\【.*?\】','',song_name)
        songs.append(song_name)
    for song in songs:
        text.insert(tk.INSERT, song)
        text.insert(tk.INSERT, '\n')
    keys=songs

def select_favor():
    global fid
    url="http://api.bilibili.com/x/v3/fav/folder/created/list-all?up_mid="+mid
    headers = {
        'User-Agent': 'apifox/1.0.0 (https://www.apifox.cn)',
        'Accept': '*/*',
        'Host': 'api.bilibili.com',
        'Connection': 'keep-alive'
    }

    response = requests.request("GET", url, headers=headers,cookies=cookie).text
    ftitles=[]
    fids=[]
    for favors in json.loads(response)['data']['list']:
        ftitles.append(favors['title'])
        fids.append(favors['id'])
    cboxf['value'] = ftitles
    cboxf.current(0)
    def func(event):
        global fid
        i=ftitles.index(cboxf.get())
        fid=fids[i]
    cboxf.bind("<<ComboboxSelected>>",func)

def from_text():
    global keys
    song_name=re.sub(r'\（.*?\）','',text.get(1.0, "end"))
    song_name=re.sub(r'\(.*?\)','',song_name)
    song_name=re.sub(r'\【.*?\】','',song_name)
    keys=song_name.split('\n')

# 登录

In [2]:
if os.path.exists('cookie.txt'):
    with open('cookie.txt')as f:
        cookie=json.load(f)
try:
    name=user()
except:
    name='请登录'
win = tk.Tk()
win.title("b站搜索收藏")
win.geometry('400x500')
tk.Label(win,text='当前用户：'+name,font=font).pack()
photo = Image.open('face.jpg').resize((300,300))
photo=ImageTk.PhotoImage(photo)
face=tk.Label(win,image=photo)
face.pack()
flag=False
tk.Button(win, text="切换用户", width=10, command=code,font=font).pack()
win.mainloop()
if flag:
    login()
    user()

# 导入歌单

In [3]:
win = tk.Tk()
win.title("导入歌单")
win.geometry('600x720')
tk.Button(win,text="查询歌单（从文本）",font=font,command=from_text).pack()
entry = tk.Entry (win,width=60)
entry.pack()
expression = tk.StringVar ()
entry ["textvariable"] = expression
button = tk.Button (win, text="查询歌单（从链接）",command=song_list,font=font)

button.pack (side="top")
text = tk.Text(win, width=60, height=40, undo=True, autoseparators=False,font=font)
text.pack()
cboxf = ttk.Combobox(win,font=font,width=20)
cboxf.pack()
tk.Button(win,text="查询收藏夹",font=font,command=select_favor).pack()

win.mainloop()

# 搜索

In [4]:
temp=260
for page in range(len(keys[temp:])//20+1):
    win = tk.Tk()
    win.geometry('1280x720')
    result=[]
    photos=[]
    button=[]
    lable=[]
    avs=[]
    select_photos=[]
    for i,key in enumerate(keys[temp+page*20:temp+(1+page)*20]):
        av,title,img=search(key,True)[0]
        download_img(img,av)
        result.append([av,title,img,key])

    cbox = ttk.Combobox(win,font=font,width=120)
    cbox.place(x=0,y=0)

    for i,[av,title,img,key] in enumerate(result):
        avs.append(av)
        photo = Image.open('./img/'+av+'.jpg').resize((256,144))
        photo=ImageTk.PhotoImage(photo)
        photos.append(photo)
        keys.append(key)
        eval('button.append(tk.Button(win,image=photos[-1],command=lambda :change("'+key+'",'+str(i)+')))')
        button[-1].grid(row=i//5+1,column=i%5,padx=23,pady=22)
        lable.append(tk.Label(win,text=title,wraplength=270,font=font,bg="#ffffff"))
        lable[-1].grid(row=i//5+1,column=i%5,sticky="s")
    tk.Button(win,text="收藏",font=font,command=lambda:favor(avs,fid,cookie)).place(x=1000,y=0)
    win.mainloop()

KeyboardInterrupt: 